# Introduction
This notebook explores how false publications in science can emerge as the product of a multi-generational simulation of science.

In [116]:
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import scipy
from scipy.stats import beta, binom, entropy
import random
import json
import copy
import math

# global variables
num_bins = 30
num_draws = 10
num_participants = 100
num_generations = 10

## Helper functions

In [117]:
# pretty print the scientific record
def print_record(scientific_record):
    print("Scientific record")
    for i in range(0, num_bins):
        print(f"   bin {i}: {scientific_record[i][0]} zero(s), {scientific_record[i][1]} one(s)")
    print()

# return 1 with a 70% probability, 0 otherwise
def random_sample():
    random_number = random.random()
    
    if random_number < 0.7:
        return 1
    else:
        return 0

# sum the posterior KL divergences between the prior and posterior 
# scientific records for only the changed bin
def kl_divergence(count_zero_p, count_one_p, count_zero_q, count_one_q):
    p_probs = [count_zero_p/(count_zero_p + count_one_p), (count_one_p)/(count_zero_p + count_one_p)]
    q_probs = [count_zero_q/(count_zero_q + count_one_q), (count_one_q)/(count_zero_q + count_one_q)]

    return scipy.stats.entropy(p_probs, q_probs)

##  Reporting Settings
A participant is in one of three settings for how they are allowed to report their data
1. **Rate**: Report bin success rate
2. **Data**: Report full results of bin samples
3. **Subset**: Report partial results of bin samples

In [118]:
class ReportingSetting:
    def __init__(self, name):
        if name not in {"rate", "data", "subset"}:
            raise ValueError("Improper setting name")
        self.name = name

## Sample bins
To gather data, scientists choose actions (draws from a given bin) that maximize their expected value of information.

In [119]:
def draw(draw_number, bin_sample_order, values_sampled, scientific_record):
    # expected value of information for drawing from each of the bins
    evis = {}
    
    for bin_num in range(0, num_bins):
        count_zero_p = scientific_record[bin_num][0]
        count_one_p = scientific_record[bin_num][1]
        
        count_zero_q = count_zero_p
        count_one_q = count_one_p
        
        for i in range(0, len(bin_sample_order)):
            if bin_sample_order[i] == bin_num:
                value_drawn = values_sampled[i]
                if value_drawn == 0:
                    count_zero_q += 1
                elif value_drawn == 1:
                    count_one_q += 1
    
        payoff_zero = kl_divergence(count_zero_p, count_one_p, count_zero_q + 1, count_one_q)
        payoff_one = kl_divergence(count_zero_p, count_one_p, count_zero_q, count_one_q + 1)
        
        prob_zero = count_zero_p / (count_zero_p + count_one_p)
        prob_one = count_one_p / (count_zero_p + count_one_p)
        evis[bin_num] = (prob_zero * payoff_zero) + (prob_one * payoff_one)
    
#         print(f"      probability of sampling a zero: {prob_zero}")
#         print(f"      probability of sampling a one: {prob_one}")
#         print(f"      payoff of drawing a zero: {payoff_zero}")
#         print(f"      payoff of drawing a one: {payoff_one}")
#         print(f"      evi for sampling from bin {bin_num}: {evis[bin_num]}\n")
    
    # choose bin with the highest EVI to sample from
    max_evi_bins = []
    max_evi_value = None
    for bin_num, evi_value in evis.items():
        if max_evi_value is None or evi_value > max_evi_value:
            max_evi_bins = [bin_num]  # start with a new list for a higher maximum
            max_evi_value = evi_value
        elif evi_value == max_evi_value:
            max_evi_bins.append(bin_num)  # add bin to list in case of a tie

    # If there are ties, choose a bin randomly from the tied bins
    chosen_bin = random.choice(max_evi_bins)
    
    return chosen_bin, random_sample()

## Choose a bin

Participants select the bin to report whose results maximize the KL divergence between the prior and posterior distributions.

In [120]:
def choose_bin(bin_sample_order, values_sampled, scientific_record):
#     print(bin_sample_order)
#     print(values_sampled)
    
    kl = {}
    
    for bin_num in range(0, num_bins):
        count_zero_p = scientific_record[bin_num][0]
        count_one_p = scientific_record[bin_num][1]
        count_zero_q = count_zero_p
        count_one_q = count_one_p
        
        for i in range(0, num_draws):
            if bin_sample_order[i] == bin_num:
                value_drawn = values_sampled[i]
                if value_drawn == 0:
                    count_zero_q += 1
                elif value_drawn == 1:
                    count_one_q += 1
                    
        kl[bin_num] = kl_divergence(count_zero_p, count_one_p, count_zero_q, count_one_q)
        
#         print(f"   kl divergence of bin {bin_num}: {kl[bin_num]}")

    # choose bin with the highest kl divergence value
    max_kl_bins = []
    max_kl_value = None
    for bin_num, kl_value in kl.items():
        if max_kl_value is None or kl_value > max_kl_value:
            max_kl_bins = [bin_num]  # start with a new list for a higher maximum
            max_kl_value = kl_value
        elif kl_value == max_kl_value:
            max_kl_bins.append(bin_num)  # add bin to list in case of a tie

    return random.choice(max_kl_bins) # choose random if there are ties for best bin

## Report findings

Participants report their findings, exaggerating their results by some degree $\alpha$. When $\alpha = 0$, this reduces to the strategy of reporting honest, unmanipulated results. When $\alpha = 1$, this reduces to the strategy of reporting maximum values.

In [121]:
class ReportingStrategy():
    def __init__(self):
        pass
    
    def report(self, reporting_setting, alpha, bin_history):
        num_zeros = bin_history.count(0)
        num_ones = bin_history.count(1)
        
        if alpha < 0 or alpha > 1:
            raise ValueError("Alpha must be between 0 and 1")
        
        # overreport by a proportion of alpha of the remaining rate to get to a value of 1
        if reporting_setting == "rate":
            if num_ones + num_zeros == 0:
                accurate_rate = 0.5
            else:
                accurate_rate = num_ones / (num_ones + num_zeros)
            return(accurate_rate + alpha * (1 - accurate_rate))
            
        # overreport the number of '1's and underreport the number of '0's by a rate of alpha 
        elif reporting_setting == "data":
            num_reported_zeros = round(num_zeros * (1 - alpha))
            num_reported_ones = round(num_ones * (1 + alpha))
            return({"0": num_reported_zeros, "1": num_reported_ones})
        
        # remove (100 * alpha)% of the '0' results
        elif reporting_setting == "subset":
            num_reported_zeros = round(num_zeros * (1 - alpha))
            return({"0": num_reported_zeros, "1": num_ones})

## Participants
Each participant has a personal records of draws and results.

In [122]:
class Participant:
    next_id = 1  # Class variable to keep track of the next available ID

    def __init__(self, alpha, reporting_setting):        
        self.id = Participant.next_id  # Assign a unique ID to the participant
        Participant.next_id += 1  # Update the next available ID for the next participant

        self.alpha = alpha                                                   # degree of exaggeration
        self.reporting_setting = reporting_setting                           # type of report they can make
        
        self.strategy_report = ReportingStrategy()                           # how to report given samples
        self.bin_sample_order = []                                           # order of bins sampled
        self.values_sampled = []                                             # values received across draws
        self.bin_choice = -1                                                 # the bin chosen to be reported
        reported_results = None                                              # the results reported
        
    def sample(self, scientific_record):
        sample_number = len(self.bin_sample_order)
        bin_number, value = draw(len(self.values_sampled), self.bin_sample_order, self.values_sampled, scientific_record)
        self.bin_sample_order.append(bin_number)
        self.values_sampled.append(value)

        return bin_number, value
        
    def choose_bin(self, scientific_record):
        bin_choice = choose_bin(self.bin_sample_order, self.values_sampled, scientific_record)
        self.bin_choice = bin_choice
        
        return bin_choice

    def report(self, alpha):
        history = get_full_history(self.bin_sample_order, self.values_sampled)
        bin_history = history[num_draws - 1][self.bin_choice]
        self.reported_results = self.strategy_report.report(self.reporting_setting.name, self.alpha, bin_history)

In [123]:
# returns a data structure that shows, on each draw, the values seen in each bin at that point
def get_full_history(bin_sample_order, values_sampled):
    history = {draw_number: {bin_number: [] for bin_number in range(num_bins)} for draw_number in range(num_draws)}

    for draw in range(len(bin_sample_order)):
        if draw == 0:
            history[draw][bin_sample_order[draw]].append(values_sampled[draw])
        else:
            prev_history = history[draw - 1].copy()
            for bin_num in prev_history:
                if bin_num == bin_sample_order[draw]:
                    history[draw][bin_num] = prev_history[bin_num] + [values_sampled[draw]]
                else:
                    history[draw][bin_num] = prev_history[bin_num][:]
    return history

## Initialize participants

In [124]:
def make_participants(setting, alpha_value):
    participants = []

    for i in range (0, num_participants):
        if setting == "rate":
            report_set = ReportingSetting("rate")
        elif setting == "data":
            report_set = ReportingSetting("data")
        elif setting == "subset":
            report_set = ReportingSetting("subset")

        # make participant
        participant = Participant(alpha=alpha_value, reporting_setting=report_set)
                        
        participants.append(participant)

    return(participants)

## Defining the peer review layer
This layer takes in reports from scientists, selects reports for publication, and thereby updates the scientific record. The scientific record consists of the number of positive and negative draws associated with each bin. We do this because we assume that published data is given fully (not just publishing some sort of aggregation of the submitted results).

In [125]:
# defines the utility that a publisher assigns to a report, when the report is 
# accompanied by data (subset or full reporting styles)
def utility_publish_data(report, scientific_record, bin_choice):    
    # bump by amount of supporting data
    score = report["0"] + report["1"]
    
    # bump by level of surprise
    count_zero_p = scientific_record[bin_choice][0]
    count_one_p = scientific_record[bin_choice][1]
    kl = kl_divergence(count_zero_p, count_one_p, count_zero_p + report["0"], count_one_p + report["1"])
    score += 10 * kl

    # bump by publication bias (+5% if positive and -5% if negative)
    if report["0"] > report["1"]:
        score = 0.95 * score
    else:
        score = 1.05 * score

#     print(f"final score {score}")
    return score

In [126]:
# defines the utility that a publisher assigns to a report, when the report
# consists soley of a rate
def utility_publish_rate(report, scientific_record, bin_choice):
    return 0

In [127]:
# TODO: update the published scientific record
# the peer review board selects reports for publication and returns the updated scientific record
def peer_review(participants, scientific_record):  
    actor_optimality = 1
    id_to_prob = {}
    total = 0
    
    for participant in participants:
        report_util = utility_publish_data(participant.reported_results, scientific_record, participant.bin_choice)
        report_prob = math.exp(actor_optimality * report_util)
        id_to_prob[participant.id] = report_prob
        total += report_prob
    
    for prob in id_to_prob:
        id_to_prob[prob] /= total
    
    # publish 20% of the submitted reports
    number_published = math.ceil(len(participants)/5)
    participant_ids = list(id_to_prob.keys())
    probabilities = list(id_to_prob.values())

    for i in range(0, number_published):
        selected_participant = random.choices(participant_ids, probabilities)[0]
#         print(f"participant selected: {selected_participant}")
        
        # update scientific record
        for p in participants:
            if p.id == selected_participant:
                scientific_record[p.bin_choice][0] += p.reported_results["0"]
                scientific_record[p.bin_choice][1] += p.reported_results["1"]
                
        # remove that participant from the list
        selected_index = participant_ids.index(selected_participant)
        del participant_ids[selected_index]
        del probabilities[selected_index]
        
        # Re-normalize the probabilities
        total_prob = sum(probabilities)
        probabilities = [prob / total_prob for prob in probabilities]

    return scientific_record

## Run an experiment

The multi-generational experiment is run, given reporting setting and exaggeration values.

In [128]:
def run_experiment(setting, alpha_value):
    # each experiment starts with a blank cannon (starts with 1-1 prior)
    scientific_record = {}
    for bin_num in range(0, num_bins):
        scientific_record[bin_num] = {} 
        scientific_record[bin_num][0] = 1
        scientific_record[bin_num][1] = 1
    
    for generation in range(0, num_generations):
        print(f"* Generation {generation}...")
        print_record(scientific_record)
        
        # each generation gets an entirely new set of participants
        participants = make_participants(setting, alpha_value)

        # scientists explore and submit reports
        for participant in participants:
            # sample
            for i in range(0, num_draws):
                bin_number, value = participant.sample(scientific_record)
                
#                 print(f"   sample from bin {bin_number}: {value}")

            # choose the bin
            bin_choice = participant.choose_bin(scientific_record)
            print(f"   chose bin {bin_choice}")

            # specify alpha value
            participant.report(alpha_value)
            
        # the peer review board selects reports for publication and returns the updated scientific record
        scientific_record = peer_review(participants, scientific_record)
        print()

In [129]:
run_experiment("data", 0)

* Generation 0...
Scientific record
   bin 0: 1 zero(s), 1 one(s)
   bin 1: 1 zero(s), 1 one(s)
   bin 2: 1 zero(s), 1 one(s)
   bin 3: 1 zero(s), 1 one(s)
   bin 4: 1 zero(s), 1 one(s)
   bin 5: 1 zero(s), 1 one(s)
   bin 6: 1 zero(s), 1 one(s)
   bin 7: 1 zero(s), 1 one(s)
   bin 8: 1 zero(s), 1 one(s)
   bin 9: 1 zero(s), 1 one(s)
   bin 10: 1 zero(s), 1 one(s)
   bin 11: 1 zero(s), 1 one(s)
   bin 12: 1 zero(s), 1 one(s)
   bin 13: 1 zero(s), 1 one(s)
   bin 14: 1 zero(s), 1 one(s)
   bin 15: 1 zero(s), 1 one(s)
   bin 16: 1 zero(s), 1 one(s)
   bin 17: 1 zero(s), 1 one(s)
   bin 18: 1 zero(s), 1 one(s)
   bin 19: 1 zero(s), 1 one(s)
   bin 20: 1 zero(s), 1 one(s)
   bin 21: 1 zero(s), 1 one(s)
   bin 22: 1 zero(s), 1 one(s)
   bin 23: 1 zero(s), 1 one(s)
   bin 24: 1 zero(s), 1 one(s)
   bin 25: 1 zero(s), 1 one(s)
   bin 26: 1 zero(s), 1 one(s)
   bin 27: 1 zero(s), 1 one(s)
   bin 28: 1 zero(s), 1 one(s)
   bin 29: 1 zero(s), 1 one(s)

   chose bin 10
   chose bin 23
   chose bi

   chose bin 6
   chose bin 6
   chose bin 20
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 20
   chose bin 27
   chose bin 6
   chose bin 6
   chose bin 27
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 27
   chose bin 6
   chose bin 20
   chose bin 6
   chose bin 20
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 20
   chose bin 6
   chose bin 27
   chose bin 20
   chose bin 6
   chose bin 27
   chose bin 20
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 27
   chose bin 6
   chose bin 27
   chose bin 6
   chose bin 6
   chose bin 20
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 20
   chose bin 6
   chose bin 20
   chose bin 6
   chose bin 27
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 6
   chose bin 20
   chose bin 27
   ch

   chose bin 29
   chose bin 25
   chose bin 29
   chose bin 29
   chose bin 16
   chose bin 25
   chose bin 0
   chose bin 0
   chose bin 23
   chose bin 23
   chose bin 0
   chose bin 23
   chose bin 11
   chose bin 25
   chose bin 0
   chose bin 11
   chose bin 0
   chose bin 0
   chose bin 29
   chose bin 25
   chose bin 0
   chose bin 16
   chose bin 11
   chose bin 29
   chose bin 29
   chose bin 25
   chose bin 16
   chose bin 29
   chose bin 29
   chose bin 23
   chose bin 25
   chose bin 0
   chose bin 0
   chose bin 29
   chose bin 23
   chose bin 23
   chose bin 0
   chose bin 29
   chose bin 25
   chose bin 16
   chose bin 11
   chose bin 16
   chose bin 29
   chose bin 16
   chose bin 16
   chose bin 11
   chose bin 16
   chose bin 11
   chose bin 23

* Generation 7...
Scientific record
   bin 0: 2 zero(s), 10 one(s)
   bin 1: 4 zero(s), 24 one(s)
   bin 2: 4 zero(s), 18 one(s)
   bin 3: 4 zero(s), 18 one(s)
   bin 4: 15 zero(s), 37 one(s)
   bin 5: 2 zero(s), 26 one(s)
  

   chose bin 21
   chose bin 21
   chose bin 21
   chose bin 21
   chose bin 26
   chose bin 21
   chose bin 21
   chose bin 21
   chose bin 21
   chose bin 21



## Compare hyperparameters across experiments

Run multiple experiments to see the effect of different reporting settings and alpha values on the published scientific record.

In [ ]:
reporting_settings = ["rate", "data", "subset"]
alpha_values = [0, 0.25, 0.5, 0.75, 1]

results = [] # TODO: fix this into whatever you want

for reporting_setting in reporting_settings:
    for alpha_value in alpha_values:
        result = run_experiment(reporting_strategy, setting, alpha_value)
        results.append(result)

        # TODO: save the results

## Analyze the results

In [ ]:
# Initialize the dictionary to store mean percent errors
mean_percent_errors_dict = {}

# Number of runs for each key
num_runs = 10

for gathering_strategy in gathering_strategies:
    for bin_choosing_strategy in bin_choosing_strategies:
        for reporting_strategy in reporting_strategies:
            for setting in reporting_setting:
                for alpha_value in alpha_values:
                    # Initialize a list to store MPE for each run
                    mpe_list = []

                    for _ in range(num_runs):
                        participants = make_participants(gathering_strategy, bin_choosing_strategy, reporting_strategy, setting, alpha_value)
                        mean_percent_error = peer_review(participants)
                        mpe_list.append(mean_percent_error)

                    # Calculate the average MPE
                    avg_mpe = np.mean(mpe_list)

                    # Create a key based on the variable names
                    key = (gathering_strategy, bin_choosing_strategy, reporting_strategy, setting, alpha_value)

                    # Store the average MPE in the dictionary
                    mean_percent_errors_dict[key] = avg_mpe

print(mean_percent_errors_dict)

In [ ]:
# Convert tuple keys to strings
string_keys_dict = {str(key): value for key, value in mean_percent_errors_dict.items()}

# Specify the file path to save the JSON file
json_file_path = 'mean_percent_errors.json'

# Save the dictionary with string keys to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(string_keys_dict, json_file)

print(f"Mean percent errors saved to {json_file_path}")

## Find best 5 and worst 5 settings

In [ ]:
# Convert tuple keys to strings
string_keys_dict = {str(key): value for key, value in mean_percent_errors_dict.items()}

# Sort the dictionary by values
sorted_dict = dict(sorted(string_keys_dict.items(), key=lambda item: item[1]))

# Print the best five settings with their MPE
print("Best 15 Settings:")
for key in list(sorted_dict)[:15]:
    setting_tuple = eval(key)  # Convert the string back to a tuple
    mpe = sorted_dict[key]
    print(f"{setting_tuple}: {mpe}")

# Print the worst five settings with their MPE
print("\nWorst 15 Settings:")
for key in list(sorted_dict)[-15:]:
    setting_tuple = eval(key)  # Convert the string back to a tuple
    mpe = sorted_dict[key]
    print(f"{setting_tuple}: {mpe}")

In [24]:
# Convert tuple keys to strings
string_keys_dict = {str(key): value for key, value in mean_percent_errors_dict.items()}

# Filter out settings where alpha is equal to 1
filtered_dict = {key: value for key, value in string_keys_dict.items() if eval(key)[-1] != 1}

# Sort the filtered dictionary by values in descending order
sorted_filtered_dict = dict(sorted(filtered_dict.items(), key=lambda item: item[1], reverse=True))
r
# Print the worst settings with alpha not equal to 1
print("Worst Settings (alpha not equal to 1) in Order:")
for key, value in sorted_filtered_dict.items():
    setting_tuple = eval(key)  # Convert the string back to a tuple
    print(f"{setting_tuple}: {value}")

NameError: name 'mean_percent_errors_dict' is not defined

In [ ]:
from collections import defaultdict

# Convert tuple keys to strings
string_keys_dict = {str(key): value for key, value in mean_percent_errors_dict.items()}

# Create a defaultdict to store MPE values for each unique setting component
component_mpes = defaultdict(list)

# Populate the defaultdict with MPE values
for key, value in string_keys_dict.items():
    setting_tuple = eval(key)
    
    # Iterate over all components in the setting tuple
    for component in setting_tuple:
        component_mpes[component].append(value)

# Calculate average MPE for each unique setting component
average_mpes = {component: np.mean(mpe_list) for component, mpe_list in component_mpes.items()}

# Print the average MPE for each setting component
print("Average MPE for Each Setting Component:")
for component, average_mpe in average_mpes.items():
    print(f"{component}: {average_mpe}")